In [1]:
%load_ext autoreload
%autoreload 2

import gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import argparse
import numpy as np
from time import sleep

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecTransposeImage, VecFrameStack, DummyVecEnv
from vizdoom.vizdoom import Mode
from vizdoom.vizdoom import ScreenFormat
from environments import doom_env
from environments import utils

In [2]:
np.random.randint(3)

0

In [3]:
model_path = r'/home/leandro/ml/rl-doom/trained_agents/deathmatch_deep_512_256-256/best_model.zip'
agent = PPO.load(model_path)
agent.policy

ActorCriticCnnPolicy(
  (features_extractor): CNNFeatureExtractor(
    (cnn): Sequential(
      (0): LayerNorm((3, 100, 156), eps=1e-05, elementwise_affine=True)
      (1): Conv2d(3, 64, kernel_size=(8, 8), stride=(4, 4), bias=False)
      (2): LayerNorm((64, 24, 38), eps=1e-05, elementwise_affine=True)
      (3): LeakyReLU(negative_slope=0.1)
      (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), bias=False)
      (5): LayerNorm((128, 11, 18), eps=1e-05, elementwise_affine=True)
      (6): LeakyReLU(negative_slope=0.1)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (8): LayerNorm((128, 9, 16), eps=1e-05, elementwise_affine=True)
      (9): LeakyReLU(negative_slope=0.1)
      (10): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=18432, out_features=512, bias=False)
      (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
  )
  (mlp_extractor): M

In [4]:
from config import EnvironmentConfig

env_config = EnvironmentConfig({
    "scenario": "bots_deathmatch_multimaps",
    "type": "singleplayer",
    "args": {
      "maps": ["M", "M_R"],
      "bots": 8,
      "curriculum": False,
      "shaping": True
    },
    "vizdoom_mode": "PLAYER",
    "n_parallel": 4,
    "frame_skip": 1,
    "frame_stack": 1,
    "action_combination": True,
    "action_noop": False,
    "obs_width": 320,
    "obs_height": 240,
    "obs_channels": 3,
    "obs_crop": [
      40,
      4,
      0,
      4
    ],
    "obs_resize": [
      0.5,
      0.5
    ]
  })

env = VecTransposeImage(
   VecFrameStack(
        DummyVecEnv([lambda: utils.create_env_with_bots(env_config, eval=True)]),
        1
    )
)

Built action space of size 18 from buttons [<Button.ATTACK: 0>, <Button.MOVE_FORWARD: 13>, <Button.MOVE_LEFT: 11>, <Button.MOVE_RIGHT: 10>, <Button.TURN_RIGHT: 14>, <Button.TURN_LEFT: 15>]
['M']
Logging with ID BD


In [5]:
import imageio
import time

frame_limit = 30*35
frame_start = 350
j = 0

for i in range(5):
    obs = env.reset()
    done = False
    images = []
    while not done:
        action, _ = agent.predict(obs, deterministic=True)

        obs, reward, done, _ = env.step(action)
        
        if j > frame_start:
            images.append(env.venv.envs[0].game.get_state().screen_buffer)
        j += 1
        
        if j > frame_start + frame_limit:
            j = 0
            break
            
        #time.sleep(0.02)

    imageio.mimsave(f'../figures/deep_deathmatch_{i}.gif', images, fps=35)

env.close()

Results:
AGENT: 0
************************
Agent BD frags: 0, deaths: 0, total reward: 0
- frag: +0.0
- damage: +0.0
- ammo: +0.0
- health: +0.0
- armor: +0.0
- distance: +0.0
************************
Results:
Jones: 0
McClane: 1
Machete: 1
MacGyver: 1
Anderson: 1
Conan: 2
Dredd: 2
Ripley: 3
AGENT: 7
************************
Agent BD frags: 7.0, deaths: 0, total reward: 16.294500000000337
- frag: +7.0
- damage: +8.0
- ammo: +0.4
- health: +0.2
- armor: +0.0
- distance: +0.6
************************
Results:
Plissken: 0
Blazkowicz: 1
Bond: 1
T800: 1
Predator: 1
MacGyver: 1
Ripley: 1
Jones: 3
AGENT: 9
************************
Agent BD frags: 9.0, deaths: 2, total reward: 18.76349999999974
- frag: +9.0
- damage: +10.1
- ammo: +0.4
- health: -1.3
- armor: +0.0
- distance: +0.5
************************
Results:
Dredd: 0
Ripley: 0
Leone: 0
Machete: 1
Blazkowicz: 1
Conan: 1
Plissken: 1
Bond: 2
AGENT: 10
************************
Agent BD frags: 10.0, deaths: 2, total reward: 20.06650000000001


In [6]:
env.close()